# trackmate_analyser.ipynb

- input data structure:
    - folder for each of three conditions: no_TRAK_77 / TRAK1_79 / TRAK2_78
    - each folder has a subfolder for each cell
    - each cell subfolder has ‘tracks.csv’, ‘edges.csv’, ‘spots.csv’
- output
    - the number of backwards and forwards tracks per cell (to be able to plot final stats)
    - a graph plotting the tracks (to check that vector is correct)
    - speeds for ‘anterograde’ and ‘retrograde’ movement in each condition
    - raw: TRACK_ID header with speeds in columns for each track
    - stacked: all TRACK_ID speeds stacked in a single column (for histogram)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import polars as pl
import pandas as pd
import seaborn as sns

## data processing

Generate the dataframe for plotting, with all the raw data across all folders + all the derived cols

In [ ]:
def zero_positions(df):
    df = df.sort('FRAME')
    return df.with_columns(POSITION_X_ZEROED = df['POSITION_X'] - df['POSITION_X'][0],
                    POSITION_Y_ZEROED = df['POSITION_Y'] - df['POSITION_Y'][0],
                    POSITION_T_ZEROED = df['POSITION_T'] - df['POSITION_T'][0])

def get_rotation_angle(condition, cell, path_to_cell_orientations):
    cell_orientations = pl.read_excel(path_to_cell_orientations, sheet_name='angles_per_cell')
    angle_df = cell_orientations.filter((pl.col('Cell') == cell) & (pl.col('Condition') == condition))
    if len(angle_df) != 1:
        return None
    else:
       return angle_df['angle (rad)'][0]

def generate_rotated_spot_positions(cell_path: str, path_to_cell_orientations) -> pd.DataFrame | None:
    for file in ['tracks.csv', 'edges.csv', 'spots.csv']:
        assert os.path.exists(os.path.join(cell_path, file))

    # From tracks.csv, we need the max distance travelled.
    tracks_df = pl.read_csv(os.path.join(cell_path, 'tracks.csv'), skip_rows_after_header=3)
    assert 'TRACK_ID' in tracks_df.columns
    spots_df = pl.read_csv(os.path.join(cell_path, 'spots.csv'), skip_rows_after_header=3)


    spots_df = spots_df[['TRACK_ID', 'POSITION_X', 'POSITION_Y', 'POSITION_Z', 'POSITION_T', 'FRAME']]
    tracks_df = tracks_df[['TRACK_ID', 'MAX_DISTANCE_TRAVELED',
                            'TRACK_MEAN_SPEED',
                            'TRACK_MAX_SPEED',
                            'TRACK_MIN_SPEED',
                            'TRACK_MEDIAN_SPEED',
                            'TRACK_STD_SPEED',
                            'MEAN_STRAIGHT_LINE_SPEED',
                            'MEAN_DIRECTIONAL_CHANGE_RATE']]
    spots_with_max_distance = spots_df.join(tracks_df, how='left', on='TRACK_ID', validate='m:1')
    spots_with_max_distance_filtered = spots_with_max_distance.sort(by=['TRACK_ID', 'FRAME'])
    if not len(spots_with_max_distance_filtered):
        print(f'warning!, no points found for cell {cell}, condition {condition}')
        return None
    spots_zeroed = spots_with_max_distance_filtered.group_by('TRACK_ID').map_groups(zero_positions)
    rotation_angle_rads = get_rotation_angle(condition, cell, path_to_cell_orientations)
    if rotation_angle_rads is None:
        print(f'warning!, no angle found for cell {cell}, condition {condition}')
        return None
    spots_rotated = spots_zeroed.with_columns(POSITION_X_ROTATED = pl.col('POSITION_X_ZEROED') * np.sin(rotation_angle_rads) -  pl.col('POSITION_Y_ZEROED') * np.cos(rotation_angle_rads),
                            POSITION_Y_ROTATED = pl.col('POSITION_X_ZEROED') * np.cos(rotation_angle_rads) +  pl.col('POSITION_Y_ZEROED') * np.sin(rotation_angle_rads))

    return spots_rotated  # ignore polars for now

def zero_positions(df):
    df = df.sort('FRAME')
    return df.with_columns(POSITION_X_ZEROED = df['POSITION_X'] - df['POSITION_X'][0],
                    POSITION_Y_ZEROED = df['POSITION_Y'] - df['POSITION_Y'][0],
                    POSITION_T_ZEROED = df['POSITION_T'] - df['POSITION_T'][0])


def get_rotation_angle(condition, cell, path_to_cell_orientations, sheet_name='Sheet1'):
    cell_orientations = pl.read_excel(path_to_cell_orientations, sheet_name=sheet_name)
    angle_df = cell_orientations.filter((pl.col('cell') == cell) & (pl.col('condition') == condition))
    if len(angle_df) != 1:
        return None
    else:
       return angle_df['angle'][0]


In [ ]:
dfs = {}
edge_dfs = {}

# generate the positions over time for each condition, cell, track. plot.
for date in ['231027', '231102', '231103', '231117']:
    path_to_tracks = f'../input_folder/tracking_results_sub_pixel/{date}'
    path_to_cell_orientations= f'{path_to_tracks}/{date}_cell_orientation_coordinates.xlsx'
    path_to_plots = f'../plots/{date}'
    if not os.path.exists(path_to_plots):
        os.mkdir(path_to_plots)
    for condition in os.listdir(path_to_tracks):
        condition_path = os.path.join(path_to_tracks, condition)
        if not os.path.isdir(condition_path):
            continue

        for cell in os.listdir(condition_path):
            cell_path = os.path.join(condition_path, cell)
            spots_rotated =  generate_rotated_spot_positions(cell_path, path_to_cell_orientations)
            dfs[(date, condition, cell)]  = spots_rotated.to_pandas()  # my polars is insufficient for what follows.    
            edge_dfs[(date, condition, cell)] = pl.read_csv(os.path.join(cell_path, 'edges.csv'), skip_rows_after_header=3).to_pandas() 

In [ ]:
all_points_df = pd.concat(dfs, keys=dfs.keys()).reset_index(names=['date', 'condition', 'cell', 'index']).drop(columns=['index'])
# information on the tracks, one row per track 
per_track = all_points_df.groupby(['date', 'condition', 'cell', 'TRACK_ID']).agg(
    max_distance_traveled=('MAX_DISTANCE_TRAVELED','first'),
    track_mean_speed=('TRACK_MEAN_SPEED','first'),
    track_max_speed=('TRACK_MAX_SPEED','first'),
    track_min_speed=('TRACK_MIN_SPEED','first'),
    track_median_speed=('TRACK_MEDIAN_SPEED','first'),
    track_std_speed=('TRACK_STD_SPEED','first'),
    mean_straight_line_speed=('MEAN_STRAIGHT_LINE_SPEED','first'),
    mean_directional_change_rate=('MEAN_DIRECTIONAL_CHANGE_RATE','first'),
    final_y_position=('POSITION_Y_ROTATED', 'last'),
).reset_index()


# as above but with edges

all_edges_df = pd.concat(edge_dfs, keys=edge_dfs.keys()).reset_index(names=['date', 'condition', 'cell', 'index']).drop(columns=['index'])
all_edges_df_with_track_info = all_edges_df.merge(per_track, how='left', on=['date', 'condition', 'cell', 'TRACK_ID'], validate='m:1')


In [ ]:
all_points_df.head()

In [ ]:
per_track.head()

## plots

Apply some filters to the tracks - then generate the various facets and aggregations we'd like

In [ ]:
all_edges_df_with_track_info

In [ ]:
all_edges_df_with_track_info.columns

In [ ]:

change_rate_filter = 10
distance_filter = 3
displacement_filter = 0

per_track_filtered = per_track[(per_track['max_distance_traveled'] > distance_filter)].copy()

all_edges_df_with_track_info['CHANGE_IN_RADS'] = all_edges_df_with_track_info['DIRECTIONAL_CHANGE_RATE'] * (all_edges_df_with_track_info['DISPLACEMENT'] / all_edges_df_with_track_info['SPEED'])
all_edges_df_with_track_info['CHANGE_IN_DEGREES'] = all_edges_df_with_track_info['CHANGE_IN_RADS'] * 180 / np.pi

all_edges_df_with_track_info_filtered = all_edges_df_with_track_info[(all_edges_df_with_track_info['max_distance_traveled'] > distance_filter) 
                                                                     & (all_edges_df_with_track_info['CHANGE_IN_DEGREES'] < change_rate_filter) 
                                                                    # &
                                                                    # (all_edges_df_with_track_info['DISPLACEMENT'] > displacement_filter)

                                                                     ].copy()

all_edges_df_with_track_info_filtered['final_y_is_above_zero'] = all_edges_df_with_track_info_filtered['final_y_position'] > 0

print(len(all_edges_df_with_track_info_filtered))

In [ ]:
print(per_track_filtered.groupby('condition').TRACK_ID.nunique())

### picking out single tracks

In [ ]:
date = '231103'
condition = 'no_TRAK_77'
cell = '231103_06'
single_cell = all_points_df.query(" date == @date & condition == @condition & cell == @cell")

for track_id in single_cell['TRACK_ID'].unique():
    # save the individual tracks to csvs
    single_track = single_cell.query("TRACK_ID == @track_id")
    if single_track.MAX_DISTANCE_TRAVELED.iloc[0] > distance_filter:
        track_path = f"tracks/{condition}/i_{track_id}.csv"
    else:
        track_path = f"tracks/{condition}/e_{track_id}.csv"
    single_track[['POSITION_X_ROTATED', 'POSITION_Y_ROTATED', 'MAX_DISTANCE_TRAVELED', 'TRACK_ID']].to_csv(track_path, index=False)

In [ ]:
date = '231103'
condition = 'no_TRAK_77'
cell = '231103_06'

def plot_cell(date, condition, cell, ax, title):
    single_cell = all_points_df.query(" date == @date & condition == @condition & cell == @cell")

    for track_id in single_cell['TRACK_ID'].unique():
        single_track = single_cell.query("TRACK_ID == @track_id")
        if single_track.MAX_DISTANCE_TRAVELED.iloc[0] > distance_filter:
            ax.plot(single_track['POSITION_X_ROTATED'], single_track['POSITION_Y_ROTATED'], alpha=1, zorder=2, linewidth=2 ) #, c='TRACK_ID')
        else:
            ax.plot(single_track['POSITION_X_ROTATED'], single_track['POSITION_Y_ROTATED'], alpha=0.35, zorder=1, c='grey') #, c='TRACK_ID')
    
    ax.set_ylim(-10, 10)
    ax.set_xlim(-8, 8)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_title(title, fontsize=30)

fig, axes = plt.subplots(ncols=3, sharey=True, figsize=(30, 10))
plot_cell('231103', 'no_TRAK_77', '231103_06', axes[0], title='no TRAK')
plot_cell('231103', 'TRAK1_79', '231103_07', axes[1], title='TRAK1 (1-702)')
plot_cell('231103', 'TRAK2_78', '231103_07', axes[2], title='TRAK2 (1-700)')
for ax in axes:
    ax.tick_params(axis='both', which='major', labelsize=30)
    ax.set_xlim(-8, 8)
    ax.set_ylim(-7, 9)
    ax.set_aspect('equal')
    ax.set_xticks([-5, 0, 5])
    ax.set_yticks([-5, 0, 5])
    ax.set_xlabel('aligned x position (µm)', fontsize=30)
axes[0].set_ylabel('aligned y position (µm)', fontsize=30)
plt.tight_layout()
plt.savefig('plots/240110.png', dpi=300, bbox_inches='tight')
plt.savefig('plots/240110.svg', bbox_inches='tight')

plt.show()


### Distribution of final y position for TRAK1 and TRAK2

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.violinplot(hue='condition', x='final_y_position', y='condition', data=per_track_filtered[per_track_filtered['condition'] != 'no_TRAK_77'], cut=0)
ax.axvline(0, color='black', linewidth=1.5)
plt.show()


In [ ]:
trak1_final_y = per_track_filtered[per_track_filtered['condition'] == 'TRAK1_79']['final_y_position'].reset_index(drop=True)
trak2_final_y = per_track_filtered[per_track_filtered['condition'] == 'TRAK2_78']['final_y_position'].reset_index(drop=True)
max_length = max(len(trak1_final_y), len(trak2_final_y))

# Reindex both series to the maximum length
trak1_final_y = trak1_final_y.reindex(range(max_length))
trak2_final_y = trak2_final_y.reindex(range(max_length))

# Create the DataFrame
final_y_comparison_table = pd.DataFrame({
    'TRAK1_79_final_y_position': trak1_final_y.values,
    'TRAK2_78_final_y_position': trak2_final_y.values
})

final_y_comparison_table.to_csv('final_y_comparison_table.csv', index=False)

In [ ]:

proportions_above_zero = per_track_filtered.groupby('condition')['final_y_position'].apply(lambda x: (x > 0).sum() / len(x))
print(proportions_above_zero)

In [ ]:
per_track_filtered[per_track_filtered['condition'] != 'no_TRAK_77']['max_distance_traveled'].describe()

In [ ]:

output_folder = '../output_folder/tracking_results_sub_pixel'

if not os.path.exists(output_folder):
    os.mkdir(output_folder)


trak1_final_y = per_track_filtered[per_track_filtered['condition'] == 'TRAK1_79']['final_y_position']
trak2_final_y = per_track_filtered[per_track_filtered['condition'] == 'TRAK2_78']['final_y_position']
final_y_comparison = pd.DataFrame({
    'TRAK1_79_final_y_position': trak1_final_y.reset_index(drop=True),
    'TRAK2_78_final_y_position': trak2_final_y.reset_index(drop=True)
})

final_y_comparison.to_csv(f'{output_folder}/final_y_comparison.csv')

### Distribution of edge speeds depending on whether the final y position is above or below zero

In [ ]:
fig, axes = plt.subplots(ncols=2, sharey=True, figsize=(10, 10))
sns.swarmplot(ax=axes[0], hue='condition', y='SPEED', x='final_y_is_above_zero', data=all_edges_df_with_track_info_filtered[all_edges_df_with_track_info_filtered['condition'] == 'TRAK1_79'])
sns.swarmplot(ax=axes[1], hue='condition', y='SPEED', x='final_y_is_above_zero', data=all_edges_df_with_track_info_filtered[all_edges_df_with_track_info_filtered['condition'] == 'TRAK2_78'])
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.boxplot(x='final_y_is_above_zero', y='SPEED', hue='condition', data=all_edges_df_with_track_info_filtered[all_edges_df_with_track_info_filtered['condition'] != 'no_TRAK_77'])

In [ ]:
up_trak1 = all_edges_df_with_track_info_filtered[(all_edges_df_with_track_info_filtered['condition'] == 'TRAK1_79') & (all_edges_df_with_track_info_filtered['final_y_is_above_zero'] == True)]['SPEED'].reset_index(drop=True)
down_trak1 = all_edges_df_with_track_info_filtered[(all_edges_df_with_track_info_filtered['condition'] == 'TRAK1_79') & (all_edges_df_with_track_info_filtered['final_y_is_above_zero'] == False)]['SPEED'].reset_index(drop=True)
up_trak2 = all_edges_df_with_track_info_filtered[(all_edges_df_with_track_info_filtered['condition'] == 'TRAK2_78') & (all_edges_df_with_track_info_filtered['final_y_is_above_zero'] == True)]['SPEED'].reset_index(drop=True)
down_trak2 = all_edges_df_with_track_info_filtered[(all_edges_df_with_track_info_filtered['condition'] == 'TRAK2_78') & (all_edges_df_with_track_info_filtered['final_y_is_above_zero'] == False)]['SPEED'].reset_index(drop=True)

# Create the DataFrame
four_condition_table = pd.DataFrame({
    'up_trak1': up_trak1,
    'down_trak1': down_trak1,
    'up_trak2': up_trak2,
    'down_trak2': down_trak2
})

four_condition_table.to_csv('speeds.csv', index=False)

In [ ]:
four_condition_table.count()

In [ ]:
all_edges_df_with_track_info_filtered[all_edges_df_with_track_info_filtered['condition'] != 'no_TRAK_77'].groupby(['condition', 'final_y_is_above_zero']).count()

In [ ]:
up_trak1 = per_track_filtered[(per_track_filtered['condition'] == 'TRAK1_79') & (per_track_filtered['final_y_position'] > 0)]['track_mean_speed'].reset_index(drop=True)
down_trak1 = per_track_filtered[(per_track_filtered['condition'] == 'TRAK1_79') & (per_track_filtered['final_y_position'] <= 0)]['track_mean_speed'].reset_index(drop=True)
up_trak2 = per_track_filtered[(per_track_filtered['condition'] == 'TRAK2_78') & (per_track_filtered['final_y_position'] > 0)]['track_mean_speed'].reset_index(drop=True)
down_trak2 = per_track_filtered[(per_track_filtered['condition'] == 'TRAK2_78') & (per_track_filtered['final_y_position'] <= 0)]['track_mean_speed'].reset_index(drop=True)

# Create the DataFrame
four_condition_track_table = pd.DataFrame({
    'up_trak1': up_trak1,
    'down_trak1': down_trak1,
    'up_trak2': up_trak2,
    'down_trak2': down_trak2
})

four_condition_track_table.to_csv('track_speeds.csv', index=False)

In [ ]:
four_condition_track_table

In [ ]:
per_track_filtered

In [ ]:
per_track_filtered.groupby('condition').count()

In [ ]:
per_track.groupby('condition').count()

In [ ]:
per_track.groupby('condition').TRACK_ID.nunique()

In [ ]:
per_track.groupby('condition').cell.nunique()

In [ ]:
all_edges_df.groupby('condition').TRACK_ID.nunique()


In [ ]:
# cells without filter per condition.
# tracks per condition without filter
 